In [11]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [1]:
from selenium import webdriver
import time
import re
import csv
import string
import os
import random
import pickle

from collections import defaultdict  
import pandas as pd

In [2]:
 links = pd.read_csv('../comedy_comparisons-2/comedy_comparisons.train.txt', 
                     names = ['left', 'right', 'funnier'], sep=',')

In [3]:
links_1000_2000_left = links.iloc[1000:2000, 0]

In [4]:
chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [8]:
def get_comments(url):
    
    d = defaultdict()
    
    driver.get(url)
    driver.maximize_window()
    time.sleep(5)
    title = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text

    d['title'] = title

    comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(7)

    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               "]+", flags=re.UNICODE)

    name_elems=driver.find_elements_by_xpath('//*[@id="author-text"]')
    comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
    num_of_names = len(name_elems)
    for j in range(num_of_names):
        username = name_elems[j].text    # .replace(",", "|")
        comment = comment_elems[j].text    # .replace(",", "|")
        d[j] = [username, comment]
            
    return d

In [11]:
tic = pd.datetime.now()

comments_dict = defaultdict()

for i in links_1000_2000_left:
    try:
        url = 'https://www.youtube.com/watch?v=' + i
        if i not in comments_dict.keys():
            comments_dict[i] = get_comments(url)
        
    except:
        
        comments_dict[i] = {}
        
    time.sleep(.5+1*random.random())
    
toc=pd.datetime.now()

In [12]:
toc-tic #7.7 hours

datetime.timedelta(seconds=8668, microseconds=53406)

In [35]:
df = pd.DataFrame.from_dict(comments_dict, orient='index', dtype=None, columns=None)

In [36]:
df.reset_index(inplace=True)
df.rename(columns={'index':'video_id'}, inplace=True)

In [37]:
df = pd.melt(df, id_vars=['video_id', 'title'])
df.rename(columns={'variable':'comment_order', 'value':'comment_'}, inplace=True)

In [43]:
df.shape

(9019, 4)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516000 entries, 0 to 515999
Data columns (total 4 columns):
video_id         516000 non-null object
title            516000 non-null object
comment_order    516000 non-null object
comment_         9019 non-null object
dtypes: object(4)
memory usage: 15.7+ MB


In [40]:
with open('comments_1000_2000_left_nans.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [42]:
df.dropna(axis=0, inplace=True)

In [20]:
df[['user', 'comment']] = pd.DataFrame(df['comment_'].tolist(), index=df.index)

In [21]:
df.drop('comment_', axis=1, inplace=True)

In [44]:
with open('comments_1000_2000_left_clean_9019.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [45]:
df

,video_id,title,comment_order,comment_
0,O5utqi_QtRU,Grape Stomp Lady - Funny Classic Accident,0,"[Ashley Casey, STEWIE!!!!]"
1,bCL9tcxBOMs,The realities of the new South Africa,0,"[TheCamshafter, You have to have lived in SA t..."
2,FPgpwNzYIrg,Wife gets pranked hard,0,"[Marlowann, It's been like two years since I'v..."
3,LTDAW-Wbf08,Stoned News Reporter,0,"[Kay Carter, Rugged soldiers burning drugs! LM..."
4,ALZZx1xmAzg,The IT Crowd - Series 2 - Episode 3: Piracy wa...,0,"[Rob Robinson737, An FBI agent in England shoo..."
...,...,...,...,...
514303,54AW7V2O9xc,CLUTCH Comedian Bryson Turner,1495,"[Borqna Iordanova, I'M COMING, I'M FROM 9GAG]"
514647,54AW7V2O9xc,CLUTCH Comedian Bryson Turner,1496,"[Descoordenadoo, damn, that diss made me snarf..."
514991,54AW7V2O9xc,CLUTCH Comedian Bryson Turner,1497,"[ZavulonPorter, WHAT DID HE SAY IN RESPONSE?? ..."
515335,54AW7V2O9xc,CLUTCH Comedian Bryson Turner,1498,"[iñigo san jose, 0:04 to 0:33 ,19 awkwards sec..."


Commets with API

In [8]:
YOUTUBE_COMMENT_URL = 'https://www.googleapis.com/youtube/v3/commentThreads'
def get_video_comment(self):

    def load_comments(self):
        for item in mat["items"]:
            comment = item["snippet"]["topLevelComment"]
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"]
            print("Comment by {}: {}".format(author, text))
            if 'replies' in item.keys():
                for reply in item['replies']['comments']:
                    rauthor = reply['snippet']['authorDisplayName']
                    rtext = reply["snippet"]["textDisplay"]

                print("\n\tReply by {}: {}".format(rauthor, rtext), "\n")

    parser = argparse.ArgumentParser()
    mxRes = 20
    vid = str()
    parser.add_argument("--c", help="calls comment function by keyword function", action='store_true')
    parser.add_argument("--max", help="number of comments to return")
    parser.add_argument("--videourl", help="Required URL for which comments to return")
    parser.add_argument("--key", help="Required API key")

    args = parser.parse_args()

    if not args.max:
        args.max = mxRes

    if not args.videourl:
        exit("Please specify video URL using the --videourl=parameter.")

    if not args.key:
        exit("Please specify API key using the --key=parameter.")

    try:
        video_id = urlparse(str(args.videourl))
        q = parse_qs(video_id.query)
        vid = q["v"][0]

    except:
        print("Invalid YouTube URL")

    parms = {
                'part': 'snippet,replies',
                'maxResults': args.max,
                'videoId': vid,
                'key': args.key
            }

    try:

        matches = self.openURL(YOUTUBE_COMMENT_URL, parms)
        i = 2
        mat = json.loads(matches)
        nextPageToken = mat.get("nextPageToken")
        print("\nPage : 1")
        print("------------------------------------------------------------------")
        load_comments(self)

        while nextPageToken:
            parms.update({'pageToken': nextPageToken})
            matches = self.openURL(YOUTUBE_COMMENT_URL, parms)
            mat = json.loads(matches)
            nextPageToken = mat.get("nextPageToken")
            print("\nPage : ", i)
            print("------------------------------------------------------------------")

            load_comments(self)

            i += 1
    except KeyboardInterrupt:
        print("User Aborted the Operation")

    except:
        print("Cannot Open URL or Fetch comments at a moment")

In [ ]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "../client_secret.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes) #
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id="gVVIIj_PFEhaFCYIlv2hlcyn"
    )
    response = request.execute()

    print(response)

if __name__ == "__main__":
    main()

In [1]:
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python
scopes = ["https://www.googleapis.com/youtube/v3/commentThreads"]


def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = "XXXXXXXXXXXX"
    client_secrets_file = "../client_secret.json"
    
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials, developerKey=DEVELOPER_KEY)
    
    

    request = youtube.commentThreads().list(
        part="snippet,replies",
        videoId="_VB39Jo8mAQ"
    )
    response = request.execute()

    print(response)

if __name__ == "__main__":
    main()